In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

# 1. import list of outside elective courses provided by UBC office

In [3]:
import pandas as pd
df = pd.read_excel(r'UBC MechEng Outside Electives Names (Atypical).xlsx', header = None)
df

,0
0,APSC 440
1,BIOC 300
2,BIOC 301
3,BIOC 302
4,CEEN 501
5,CHBE 355
6,CHBE 477
7,CHBE 483
8,CHBE 484
9,CHBE 485


In [4]:
course_codes = list(df[0])

In [5]:
len(list(dict.fromkeys(course_codes)))

29

# 2. Use the course codes to find and scrape corresponding course names and descriptions

In [6]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC",
    "PHYS": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=PHYS",
    "MATH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=12&code=MATH",
    "MINE": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MINE",
    "MTRL": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MTRL",
    "IGEN": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=IGEN",
    "COMR": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=4&code=COMR",
    "CHBE": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=CHBE",
    "CEEN": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=CEEN",
    "BIOC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=10&code=BIOC"
}

In [7]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  APSC 440
Cannot find  BIOC 300
scraped  BIOC 301
scraped  BIOC 302
scraped  CEEN 501
scraped  CHBE 355
scraped  CHBE 477
scraped  CHBE 483
scraped  CHBE 484
scraped  CHBE 485
scraped  COMR 329
scraped  COMR 457
scraped  COMR 458
scraped  COMR 465
scraped  COMR 473
scraped  IGEN 450
scraped  IGEN 451
scraped  IGEN 452
Cannot find  IGEN 453
scraped  MATH 300
scraped  MATH 400
scraped  MINE 482
scraped  MTRL 494
scraped  MTRL 495
scraped  MTRL 496
scraped  PHYS 404
scraped  PHYS 405
scraped  PHYS 438
scraped  APSC 486
27


# 3. Inspect collected data and write to CSV

In [8]:
#2 courses did not exist on website
course_codes = [code for code in course_codes if code != "BIOC 300" and code != "IGEN 453"]

In [9]:
course_names

['Management Fundamentals for Technology-Based Product Marketing and Development',
 'Biochemistry Laboratory',
 'General Biochemistry',
 'Energy System Fundamentals',
 'Kinetics and Reactor Design',
 'Fuel Cell and Electrochemical Engineering',
 'Energy Engineering',
 'Green Engineering Principles and Applications for Process Industries',
 'Air Pollution Prevention and Control',
 'Principles of Organizational Behaviour',
 'Fundamentals of Financial Accounting',
 'Fundamentals of Managerial Accounting',
 'Marketing Management',
 'Business Finance',
 'Pipeline Engineering',
 'Pipeline Systems and Infrastructure',
 'Pipeline Design',
 'Introduction to Complex Variables',
 'Applied Partial Differential Equations',
 'Maintenance Engineering',
 'Composite Materials',
 'Biomaterials',
 'Materials Sustainability',
 'Introduction to Medical Physics',
 'Radiation Biophysics',
 'Zoological Physics',
 'New Venture Design']

In [10]:
course_descs

['Management topics are presented from the perspective of technology-based industrial practice; project management, marketing and marketing planning, product development and commercialization, introduction to quality management, teamwork and effective individual participation. This course is not eligible for Credit/D/Fail grading. [3-0-0] Prerequisite: Fourth-year standing in Engineering.',
 'Techniques by which the chemical and physical properties of fundamental components of the cell are studied. [0-3-1] Corequisite: One of BIOC 300, BIOC 302, BIOC 303.',
 'Metabolic reactions of lipids, steroids, amino acids and nucleotides; the biochemistry of replication, transcription and translation. Credit will be granted for only one of BIOC 302 or BIOC 303. [3-0-1] Prerequisite: One of BIOL 201, BIOC 202, BIOC 203 and one of CHEM 203, CHEM 223, CHEM 233. Equivalency: BIOC 303.',
 'Thermodynamic analysis of energy conversion processes, power cycles and refrigeration cycles. Exergy analysis of 

In [11]:
df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,APSC 440,Management Fundamentals for Technology-Based P...,Management topics are presented from the persp...
1,BIOC 301,Biochemistry Laboratory,Techniques by which the chemical and physical ...
2,BIOC 302,General Biochemistry,"Metabolic reactions of lipids, steroids, amino..."
3,CEEN 501,Energy System Fundamentals,Thermodynamic analysis of energy conversion pr...
4,CHBE 355,Kinetics and Reactor Design,Kinetics of homogeneous chemical and biologica...
5,CHBE 477,Fuel Cell and Electrochemical Engineering,"Stoichiometry, thermodynamics and kinetics of ..."
6,CHBE 483,Energy Engineering,Supply and use of conventional and alternative...
7,CHBE 484,Green Engineering Principles and Applications ...,"Pollution prevention, cleaner production, gree..."
8,CHBE 485,Air Pollution Prevention and Control,"Impacts of air pollutants on health, visibilit..."
9,COMR 329,Principles of Organizational Behaviour,An introductory examination of work organizati...


In [12]:
df.to_csv('UBC_MechEng_Outside_Electives_Courses_(Atypical).csv', index = False)